In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import seaborn as sns

# 그래프 설정
# plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['font.size'] = 16
plt.rcParams['figure.figsize'] = 20, 10
plt.rcParams['axes.unicode_minus'] = False

In [2]:
from konlpy.tag import Okt
from collections import Counter
import pytagcloud
from IPython.display import Image
import re

pygame 2.0.1 (SDL 2.0.14, Python 3.8.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


### 데이터를 읽어온다.

In [3]:
df = pd.read_csv('son.csv')
df.head()

,title,description
0,"한국축구 캡틴 <b>손흥민</b>, 생수 '마신다'에서 만난다",주장 <b>손흥민</b> 선수가 소속된 프리미어리그 토트넘 홋스퍼 스페셜 패키지를 ...
1,"동아오츠카 생수 '마신다', <b>손흥민</b> 담은 토트넘 패키지 출시",생수 '마신다'가 <b>손흥민</b> 선수가 소속된 프리미어리그 토트넘 홋스퍼 스페...
2,"햄스트링에 쓰러진 <b>손흥민</b>, 토트넘은 먹구름 잔뜩",<b>손흥민</b>(토트넘)이 중요한 시점에서 부상에 쓰러졌다. 토트넘에 비상이 걸...
3,'햄스트링 부상' <b>손흥민</b> 빈자리를 누가 채울까,햄스트링 부상을 당한 <b>손흥민</b>의 빈자리를 메워야 하는 토트넘이다. <b>...
4,"부상 <b>손흥민</b>, 한일전 뛰기 어려울 듯",<b>손흥민</b>(29·토트넘)이 햄스트링 부상으로 당분간 결장할 것으로 예상된다...


### 결측치 확인

In [4]:
df.isna().sum()

title          0
description    1
dtype: int64

In [5]:
# 결측치 제거
df.dropna(inplace=True)
df.isna().sum()

title          0
description    0
dtype: int64

### 한글 정제

In [6]:
# 한글외의 문자는 모두 제거한다.
def text_clearing(text):
    # 한글 정규식
    hangul = re.compile('[^ ㄱ-ㅣ가-힣]+')
    
    # 한글외의 모든 문자를 길이가 0인 문자열로 변환한다.
    result = hangul.sub('',text)
    
    return result

In [7]:
# 테스트
a1 = df['description'][0]
print(a1)
print('-'*50)
print(text_clearing(a1))

주장 <b>손흥민</b> 선수가 소속된 프리미어리그 토트넘 홋스퍼 스페셜 패키지를 선보인다고 16일 밝혔다. 구단과 공식 라이선스 계약을 통해 제작·공개된 이번 스페셜 패키지는 <b>손흥민</b> 선수와 해리 케인, 개러스 베일, 해리... 
--------------------------------------------------
주장 손흥민 선수가 소속된 프리미어리그 토트넘 홋스퍼 스페셜 패키지를 선보인다고 일 밝혔다 구단과 공식 라이선스 계약을 통해 제작공개된 이번 스페셜 패키지는 손흥민 선수와 해리 케인 개러스 베일 해리 


In [8]:
# 한글을 정제한다.
df['title'] = df['title'].apply(lambda x : text_clearing(x))
df['description'] = df['description'].apply(lambda x : text_clearing(x))

df.head()

,title,description
0,한국축구 캡틴 손흥민 생수 마신다에서 만난다,주장 손흥민 선수가 소속된 프리미어리그 토트넘 홋스퍼 스페셜 패키지를 선보인다고 일...
1,동아오츠카 생수 마신다 손흥민 담은 토트넘 패키지 출시,생수 마신다가 손흥민 선수가 소속된 프리미어리그 토트넘 홋스퍼 스페셜 패키지를 선보...
2,햄스트링에 쓰러진 손흥민 토트넘은 먹구름 잔뜩,손흥민토트넘이 중요한 시점에서 부상에 쓰러졌다 토트넘에 비상이 걸렸다 손흥민은 일 ...
3,햄스트링 부상 손흥민 빈자리를 누가 채울까,햄스트링 부상을 당한 손흥민의 빈자리를 메워야 하는 토트넘이다 손흥민만큼은 아닐지라...
4,부상 손흥민 한일전 뛰기 어려울 듯,손흥민토트넘이 햄스트링 부상으로 당분간 결장할 것으로 예상된다 일 열리는 한국과 일...


### 형태소 분석

In [9]:
# 말뭉치 생성 (모든 문장을 하나로 합치는 작업)
title_corpus = ''.join(df['title'].tolist())
description_corpus = ''.join(df['description'].tolist())

# print(title_corpus)
# print(description_corpus)

In [10]:
# 형태소 분석
tagger = Okt()

title_nouns = tagger.nouns(title_corpus)
description_nouns = tagger.nouns(description_corpus)

# print(title_nouns)
# print(description_nouns)

In [11]:
# 단어들의 빈도수를 구한다.
title_count = Counter(title_nouns)
description_count = Counter(description_nouns)

# print(title_count)
# print(description_count)

### 키워드 다듬기
- 한글자인 데이터 삭제
- 불용어 데이터 삭제 (새로운 딕셔너리에 담아주는 작업)

In [12]:
# 한글자를 제외한 나머지를 새로운 딕셔너리에 담아준다.
title_dict = {}

for key in title_count:
    if len(key) > 1:
        title_dict[key] = title_count[key]
        
description_dict = {}

for key in description_count:
    if len(key) > 1:
        description_dict[key] = description_count[key]
        
        
remove_title_count = Counter(title_dict)
remove_title_description = Counter(description_dict)

# print(remove_title_count)
# print(remove_title_description)

In [13]:
# 불용어 제거

# 불용어 데이터 불러오기
with open('data10/korean_stopwords.txt', encoding='utf-8') as fp:
    stopwords = fp.readlines()
    
stopword = [x.strip() for x in stopwords]
# stopword

In [14]:
# 불용어를 제외한 나머지만 담아준다.
title_dict = {}

for key in remove_title_count:
    if key not in stopword:
        title_dict[key] = remove_title_count[key]

description_dict = {}
for key in remove_title_description:
    if key not in stopword:
        description_dict[key] = remove_title_description[key]
        
remove_title_count = Counter(title_dict)
remove_title_description = Counter(description_dict)

In [15]:
# 손흥민 키워드 제거 (손흥민으로 검색했기 때문에)
del remove_title_count['손흥민']
del remove_title_description['손흥민']

In [17]:
# print(remove_title_count)
# print(remove_title_description)

### 워드 클라우드로 시각화

In [19]:
# title에서 빈도수가 많은 100개의 단어를 추출한다.
rank_title_tags = remove_title_count.most_common(100)

# 워드 클라우드를 위한 단어사전을 생성한다.
title_taglist = pytagcloud.make_tags(rank_title_tags)

# 워드 클라우드 이미지를 생성한다.
pytagcloud.create_tag_image(title_taglist,'son_title.jpg', size=(450,300),
                           fontname='NanumGothic', rectangular=False)

AttributeError: Invalid font name. Should be one of Nobile, Old Standard TT, Cantarell, Reenie Beanie, Cuprum, Molengo, Neucha, Philosopher, Yanone Kaffeesatz, Cardo, Neuton, Inconsolata, Crimson Text, Josefin Sans, Droid Sans, Lobster, IM Fell DW Pica, Vollkorn, Tangerine, Coustard, PT Sans Regular

In [ ]:
# title에서 빈도수가 많은 100개의 단어를 추출한다.
rank_description_tags = remove_description_count.most_common(100)

# 워드 클라우드를 위한 단어사전을 생성한다.
description_taglist = pytagcloud.make_tags(rank_description_tags)

# 워드 클라우드 이미지를 생성한다.
pytagcloud.create_tag_image(description_taglist,'son_description.jpg', size=(450,300),
                           fontname='NanumGothic', rectangular=False)

In [ ]:
# 이미지 표시
Image(filename='son_title.jpg')

In [31]:
import wordcloud
import matplotlib.pyplot as plt
 
wordcloud = wordcloud(font_path='font/NanumGothic.ttf', background_color='white').generate(remove_title_count)

TypeError: 'module' object is not callable